In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import time
import csv
import numpy as np
import math

In [2]:
df_train = pd.read_csv("df_train.csv")
df_test = pd.read_csv("df_test.csv")
print(df_train.dtypes)

Unnamed: 0                    int64
uid                           int64
age_group                     int64
gender                        int64
city                          int64
prodName                      int64
ramCapacity                 float64
ramLeftRation               float64
romCapacity                 float64
romLeftRation               float64
color                         int64
fontSize                    float64
ct                            int64
carrier                       int64
os                            int64
bootTimes                     int64
AFuncTimes                  float64
BFuncTimes                  float64
CFuncTimes                  float64
DFuncTimes                  float64
EFuncTimes                  float64
FFuncTimes                  float64
FFuncSum                      int64
appId                         int64
usage_cnt                   float64
usage_appid_cnt             float64
usage_date_cnt              float64
duration_mean               

In [3]:
print(df_test.dtypes)

Unnamed: 0                    int64
uid                           int64
gender                        int64
city                          int64
prodName                      int64
ramCapacity                 float64
ramLeftRation               float64
romCapacity                 float64
romLeftRation               float64
color                         int64
fontSize                    float64
ct                            int64
carrier                       int64
os                            int64
bootTimes                     int64
AFuncTimes                  float64
BFuncTimes                  float64
CFuncTimes                  float64
DFuncTimes                  float64
EFuncTimes                  float64
FFuncTimes                  float64
FFuncSum                      int64
appId                         int64
usage_cnt                   float64
usage_appid_cnt             float64
usage_date_cnt              float64
duration_mean               float64
duration_max                

In [4]:
class PointDataset(Dataset):
    def __init__(self, df, transform=None, train=True):
        '''
        Args:
            csv_file(string): path of label file
            transform (callable, optional): Optional transform to be applied
                on a sample.
        '''
        self.dataframe = df
        print(self.dataframe.dtypes)
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        x = self.dataframe.iloc[idx, 3:35]
        x = x.astype('float')
        x = (x - x.mean()) / (x.std())
        #print(x)
        point = np.array(x)
        label = int(self.dataframe.iloc[idx, 2])
        label = label - 1
        if self.transform is not None:
            point = self.transform(point)
        sample = {'point': point, 'label': label}
        return sample

In [5]:
class Network(nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        '''
        Args:
            n_feature(int): size of input tensor
            n_hidden(int): size of hidden layer 
            n_output(int): size of output tensor
        '''
        super(Network, self).__init__()
        # define a liner layer
        self.hidden1 = nn.Linear(n_feature, n_hidden)
        # define sigmoid activation 
        self.sigmoid1 = nn.Sigmoid()
        self.sig = nn.Sigmoid()
        self.predict = nn.Linear(n_hidden, n_output)

    def forward(self, x):
        '''
        x(tensor): inputs of the network
        '''
        # hidden layer
        h1 = self.hidden1(x)
        # activate function
        h2 = self.sigmoid1(h1)
        # output layer
        out = self.predict(h2)
        #out = nn.functional.log_softmax(h8) + 1e-10
        '''
        Linear classifier often follows softmax to output probability,
        however the loss function CrossEntropy we used have done this 
        operation, so we don't use softmax function here.
        '''
        return out

In [6]:
def predict(classifier_net, testloader, device):
#     correct = [0 for i in range(3)]
#     total = [0 for i in range(3)]
    correct = 0
    total = 0
    
    with torch.no_grad():
        '''
        you can also stop autograd from tracking history on Tensors with .requires_grad=True 
        by wrapping the code block in with torch.no_grad():
        '''
        for data in testloader:
            point, label = data['point'], data['label']
            point, label = point.to(device).to(torch.float32), label.to(device)
            outputs = classifier_net(point)
            '''
            if you want to get probability of the model prediction,
            you can use softmax function here to transform outputs to probability.
            '''
            # transform the prediction to one-hot form
            _, predicted = torch.max(outputs, 1)
            print('model prediction: ', predicted)
            print('ground truth:', label, '\n')
            correct += (predicted == label).sum()
            total += label.size(0)
            print('current correct is:', correct.item())
            print('current total is:', total)
            
        print('the accuracy of the model is {0:5f}'.format(correct.item()/total))
        
    return correct.item() / total

In [7]:
def clip_gradient(optimizer, grad_clip):
    """
    Clips gradients computed during backpropagation to avoid explosion of gradients.

    :param optimizer: optimizer with the gradients to be clipped
    :param grad_clip: clip value
    """
    for group in optimizer.param_groups:
        for param in group["params"]:
            if param.grad is not None:
                param.grad.data.clamp_(-grad_clip, grad_clip)

In [8]:
def show_running_loss(running_loss):
    # generate x value
    x = np.array([i for i in range(len(running_loss))])
    # generate y value
    y = np.array(running_loss)
    # define a graph
    plt.figure()
    # generate curve
    plt.plot(x, y, c='b')
    # show axis
    plt.axis()
    # define title
    plt.title('loss curve:')
    #define the name of x axis
    plt.xlabel('step')
    plt.ylabel('loss value')
    # show graph
    plt.show()

In [9]:
def predict(classifier_net, testloader, device):
#     correct = [0 for i in range(3)]
#     total = [0 for i in range(3)]
    correct = 0
    total = 0
    
    with torch.no_grad():
        '''
        you can also stop autograd from tracking history on Tensors with .requires_grad=True 
        by wrapping the code block in with torch.no_grad():
        '''
        for data in testloader:
            point, label = data['point'], data['label']
            point, label = point.to(device).to(torch.float32), label.to(device)
            outputs = classifier_net(point)
            '''
            if you want to get probability of the model prediction,
            you can use softmax function here to transform outputs to probability.
            '''
            # transform the prediction to one-hot form
            _, predicted = torch.max(outputs, 1)
            print('model prediction: ', predicted)
            print('ground truth:', label, '\n')
            correct += (predicted == label).sum()
            total += label.size(0)
            print('current correct is:', correct.item())
            print('current total is:', total)
            
        print('the accuracy of the model is {0:5f}'.format(correct.item()/total))
        
    return correct.item() / total

In [10]:
def show_accuracy(running_accuracy):
    x = np.array([i for i in range(len(running_accuracy))])
    y = np.array(running_accuracy)
    plt.figure()
    plt.plot(x, y, c='b')
    plt.axis()
    plt.title('accuracy curve:')
    plt.xlabel('step')
    plt.ylabel('accuracy value')
    plt.show()

In [11]:
def train(classifier_net, trainloader, trainframe, device, lr, optimizer):
    '''
    Args:
        classifier_net(nn.model): train model
        trainloader(torch.utils.data.DateLoader): train loader
        testloader(torch.utils.data.DateLoader): test loader
        device(torch.device): the evironment your model training
        LR(float): learning rate
    '''
    # loss function
    criterion = nn.CrossEntropyLoss().to(device)
    
    optimizer = optimizer
    
    # save the mean value of loss in an epoch
    running_loss = []
    
    running_accuracy = []
    
    # count loss in an epoch 
    temp_loss = 0.0
    
    # count the iteration number in an epoch
    iteration = 0 

    for epoch in range(epoches):
        
        '''
        adjust learning rate when you are training the model
        '''
        
        # adjust learning rate
        if epoch % 20 == 0 and epoch != 0:
            LR = LR * 0.5
            for param_group in optimizer.param_groups:
                param_group['lr'] = LR

        for i, data in enumerate(trainloader):
            point, label = data['point'], data['label']
            point, label = point.to(device).to(torch.float32), label.to(device)
            outputs = classifier_net(point)
            
            
            optimizer.zero_grad()
            loss = criterion(outputs, label)
            #if loss > 0.0 and loss < 2.0:
            loss.backward()
            
            
            if grad_clip is not None:
                clip_gradient(optimizer, grad_clip)
            optimizer.step()
            temp_loss += loss.item()
            iteration +=1 
            if iteration % 1000 == 0:
                print('[{0:d},{1:5.0f}] loss {2:.5f}'.format(epoch + 1, i, loss.item()))
            #time.sleep(1)
            #for parameters in classifier_net.parameters():
                #print(parameters)
        running_loss.append(temp_loss / iteration)
        temp_loss = 0
        iteration = 0
        print('test {}:----------------------------------------------------------------'.format(epoch))
        
        # call test function and return accuracy
        vali = trainframe.sample(n=20000, replace=False, axis=0)
        valiset = PointDataset(vali, torch.tensor)
        testloader = DataLoader(dataset=valiset, batch_size=200, shuffle=False)
        running_accuracy.append(predict(classifier_net, testloader, device))
        torch.save(classifier_net, './singlelayerclassifier_params.pt')
    
    # show loss curve
    show_running_loss(running_loss)
    
    # show accuracy curve
    show_accuracy(running_accuracy)
    
    return classifier_net

In [12]:
batch_size = 16
transform = torch.tensor
device = torch.device('cuda:0')
trainset = PointDataset(df_train, transform=transform)

trainloader = DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True)

Unnamed: 0                    int64
uid                           int64
age_group                     int64
gender                        int64
city                          int64
prodName                      int64
ramCapacity                 float64
ramLeftRation               float64
romCapacity                 float64
romLeftRation               float64
color                         int64
fontSize                    float64
ct                            int64
carrier                       int64
os                            int64
bootTimes                     int64
AFuncTimes                  float64
BFuncTimes                  float64
CFuncTimes                  float64
DFuncTimes                  float64
EFuncTimes                  float64
FFuncTimes                  float64
FFuncSum                      int64
appId                         int64
usage_cnt                   float64
usage_appid_cnt             float64
usage_date_cnt              float64
duration_mean               

In [13]:
verifyset = PointDataset(df_vali, transform=transform)
testloader = DataLoader(dataset=verifyset, batch_size=batch_size, shuffle=False)

NameError: name 'df_vali' is not defined

In [14]:
classifier_net = Network(32, 256, 6).to(device)
for parameters in classifier_net.parameters():
                print(parameters)

Parameter containing:
tensor([[ 0.1339, -0.1654,  0.1370,  ...,  0.0484,  0.1174, -0.0466],
        [-0.1239, -0.0604,  0.0977,  ..., -0.0351,  0.0280,  0.0102],
        [-0.1577,  0.1440,  0.1713,  ...,  0.0935, -0.0930, -0.0494],
        ...,
        [-0.1117, -0.0350, -0.0648,  ...,  0.1231, -0.1126,  0.1749],
        [-0.0058, -0.0979,  0.0201,  ...,  0.1097, -0.1423,  0.1088],
        [ 0.1408,  0.1064,  0.0896,  ..., -0.1189,  0.1458,  0.1026]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.0965,  0.1136,  0.1597,  0.0886, -0.0386,  0.1365,  0.1661,  0.0832,
         0.0550, -0.1293,  0.0389, -0.0096, -0.0855,  0.0897,  0.1032,  0.1006,
        -0.0368, -0.1268, -0.1191, -0.1493,  0.0059,  0.0057, -0.0878, -0.0886,
        -0.0123,  0.1280,  0.0769, -0.0809,  0.1303, -0.1073, -0.1537, -0.0728,
         0.1457, -0.0741, -0.1767, -0.0782,  0.1580, -0.0971, -0.0839, -0.1064,
        -0.0448,  0.0957, -0.0699, -0.0232, -0.1479,  0.1371,  0.1027, -0.0911

In [17]:
classifier_net = torch.load('./singlelayerclassifier_params.pt')

In [14]:
lr = 1e-3
device = torch.device('cuda:0')
optimizer = optim.SGD(classifier_net.parameters(), lr=lr, momentum=0.9)
optimizeradam = optim.Adam(classifier_net.parameters(), lr=lr, betas=(0.9, 0.99))
optimizerrprop = optim.RMSprop(classifier_net.parameters(), lr=lr, alpha=0.9)

In [18]:
epoches = 20
grad_clip = 5

classifier_net.to(device)

Network(
  (hidden1): Linear(in_features=32, out_features=256, bias=True)
  (sigmoid1): Sigmoid()
  (sig): Sigmoid()
  (predict): Linear(in_features=256, out_features=6, bias=True)
)

In [ ]:
classifier_net = train(classifier_net, trainloader, df_train, device, lr, optimizeradam)

[1,  999] loss 1.23735
[1, 1999] loss 1.52438
[1, 2999] loss 1.47439
[1, 3999] loss 1.45882
[1, 4999] loss 1.51009
[1, 5999] loss 1.32447
[1, 6999] loss 1.42141
[1, 7999] loss 1.55200
[1, 8999] loss 1.47349
[1, 9999] loss 1.53165
[1,10999] loss 1.36713
[1,11999] loss 1.47849
[1,12999] loss 1.40055
[1,13999] loss 1.27575
[1,14999] loss 1.51449
[1,15999] loss 1.73469
[1,16999] loss 1.31399
[1,17999] loss 1.52230
[1,18999] loss 1.55356
[1,19999] loss 1.55203
[1,20999] loss 1.84964
[1,21999] loss 1.57876
[1,22999] loss 1.60616
[1,23999] loss 1.49156
[1,24999] loss 1.52162
[1,25999] loss 1.34184
[1,26999] loss 1.60308
[1,27999] loss 1.48436
[1,28999] loss 1.42805
[1,29999] loss 1.53952
[1,30999] loss 1.30315
[1,31999] loss 1.39780
[1,32999] loss 1.42125
[1,33999] loss 1.50766
[1,34999] loss 2.09867
[1,35999] loss 1.56041
[1,36999] loss 1.65520
[1,37999] loss 1.67495
[1,38999] loss 1.23339
[1,39999] loss 1.56482
[1,40999] loss 1.46518
[1,41999] loss 1.22431
[1,42999] loss 2.21840
[1,43999] l

        3, 5, 2, 1, 1, 3, 3, 5], device='cuda:0') 

current correct is: 231
current total is: 600
model prediction:  tensor([2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 3, 2, 3, 2, 2, 2, 1, 3, 2, 2, 3, 3, 5, 3,
        2, 5, 2, 2, 2, 3, 3, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2,
        3, 3, 3, 1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 2, 3, 2, 2, 2, 1, 2, 1, 3, 2, 2,
        2, 3, 3, 2, 2, 2, 1, 1, 3, 1, 1, 2, 2, 2, 3, 3, 1, 2, 2, 2, 2, 2, 2, 1,
        2, 3, 3, 2, 1, 2, 3, 3, 3, 2, 3, 3, 1, 3, 2, 2, 3, 3, 2, 3, 2, 3, 2, 3,
        2, 2, 2, 2, 2, 1, 3, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 1, 1, 1, 3, 2, 1,
        1, 2, 3, 2, 2, 1, 2, 3, 2, 2, 2, 2, 1, 1, 2, 1, 4, 3, 3, 1, 3, 3, 2, 3,
        3, 3, 1, 2, 3, 2, 5, 3, 2, 2, 2, 3, 2, 2, 2, 1, 2, 2, 3, 3, 2, 3, 2, 1,
        2, 4, 2, 2, 2, 3, 2, 2], device='cuda:0')
ground truth: tensor([3, 4, 1, 3, 2, 1, 1, 1, 2, 3, 3, 1, 3, 2, 1, 5, 2, 4, 4, 3, 4, 4, 5, 3,
        2, 2, 3, 2, 3, 3, 1, 1, 1, 2, 2, 3, 2, 1, 1, 2, 1, 2, 2, 2, 5, 2, 1, 2,
        2, 2, 4, 2,

        1, 4, 4, 5, 1, 4, 4, 3], device='cuda:0') 

current correct is: 652
current total is: 1800
model prediction:  tensor([2, 2, 3, 2, 3, 1, 2, 4, 3, 2, 2, 2, 1, 2, 1, 2, 2, 2, 3, 1, 2, 1, 3, 2,
        2, 2, 2, 2, 2, 3, 1, 2, 2, 4, 3, 3, 3, 2, 3, 1, 3, 2, 3, 3, 1, 2, 1, 1,
        3, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 1, 1, 2, 2, 3, 1, 3, 1, 3, 5, 2, 3,
        1, 5, 2, 2, 2, 2, 5, 2, 1, 2, 3, 2, 1, 1, 3, 2, 2, 1, 2, 1, 2, 1, 2, 2,
        1, 2, 2, 2, 4, 3, 2, 3, 3, 2, 2, 3, 1, 2, 3, 2, 1, 2, 2, 3, 2, 1, 3, 2,
        3, 2, 2, 2, 3, 4, 2, 2, 1, 1, 2, 3, 1, 1, 3, 2, 2, 2, 3, 2, 2, 2, 2, 3,
        2, 2, 2, 3, 2, 1, 3, 1, 3, 1, 4, 3, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 1, 1,
        1, 2, 2, 1, 3, 1, 2, 3, 2, 3, 4, 2, 3, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1,
        5, 2, 2, 2, 2, 2, 1, 3], device='cuda:0')
ground truth: tensor([1, 3, 4, 3, 2, 4, 1, 4, 5, 2, 2, 2, 3, 2, 4, 1, 3, 2, 4, 1, 3, 3, 3, 4,
        1, 1, 3, 4, 2, 4, 4, 5, 2, 5, 4, 1, 4, 3, 4, 4, 3, 2, 4, 4, 1, 1, 1, 1,
        4, 4, 1, 0

        2, 1, 3, 4, 2, 2, 5, 3], device='cuda:0') 

current correct is: 1063
current total is: 3000
model prediction:  tensor([3, 2, 1, 2, 2, 1, 2, 1, 3, 3, 3, 2, 1, 3, 2, 2, 1, 2, 2, 3, 2, 4, 2, 3,
        2, 3, 1, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 1, 2, 3, 2, 2, 2, 2, 3, 2,
        1, 1, 2, 2, 2, 2, 3, 2, 2, 1, 2, 2, 1, 2, 3, 2, 3, 1, 3, 3, 2, 2, 2, 1,
        2, 2, 1, 2, 2, 2, 1, 3, 2, 1, 3, 2, 2, 2, 2, 1, 2, 1, 2, 3, 2, 3, 4, 4,
        3, 2, 3, 4, 2, 2, 1, 2, 3, 2, 3, 2, 2, 3, 1, 3, 2, 2, 2, 2, 3, 3, 2, 4,
        1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 4, 2, 3, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2,
        2, 3, 2, 2, 2, 1, 1, 3, 3, 3, 2, 2, 2, 3, 2, 2, 2, 2, 3, 4, 2, 2, 1, 1,
        1, 2, 1, 1, 2, 2, 2, 1, 2, 4, 2, 3, 2, 2, 2, 2, 2, 2, 1, 2, 1, 3, 2, 2,
        2, 2, 2, 2, 1, 3, 3, 1], device='cuda:0')
ground truth: tensor([5, 2, 0, 2, 1, 2, 4, 2, 1, 5, 5, 3, 1, 2, 4, 2, 1, 1, 2, 5, 1, 4, 2, 1,
        0, 3, 1, 2, 4, 3, 3, 5, 1, 3, 3, 3, 3, 1, 3, 1, 1, 3, 2, 1, 2, 1, 2, 2,
        3, 2, 4, 

        1, 2, 3, 3, 3, 4, 4, 2], device='cuda:0') 

current correct is: 1489
current total is: 4200
model prediction:  tensor([2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 3, 3, 1, 4, 2, 3, 2, 3, 3, 3, 4, 2, 3, 3,
        1, 3, 2, 2, 2, 2, 3, 1, 3, 1, 2, 2, 1, 1, 2, 3, 1, 2, 2, 2, 2, 3, 3, 2,
        3, 2, 1, 2, 3, 1, 3, 3, 3, 3, 4, 3, 2, 2, 3, 2, 2, 3, 2, 3, 2, 2, 2, 4,
        3, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 3, 2, 2, 2, 2, 1, 2, 2, 3, 2, 4, 3, 2,
        2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 3, 2, 3, 2, 2, 3, 3, 3, 2, 2, 2, 2,
        3, 2, 4, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 1, 3, 3, 2, 2, 3, 3, 2, 2,
        1, 2, 3, 2, 3, 1, 2, 4, 3, 2, 2, 2, 2, 1, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2,
        1, 3, 3, 1, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 3, 3, 3, 2, 2, 3, 2, 3, 2,
        1, 1, 1, 2, 2, 2, 3, 3], device='cuda:0')
ground truth: tensor([4, 3, 2, 3, 4, 3, 2, 4, 2, 2, 2, 2, 2, 3, 2, 1, 2, 2, 4, 1, 2, 4, 1, 3,
        2, 5, 2, 3, 3, 2, 2, 1, 3, 1, 3, 2, 4, 2, 2, 1, 1, 3, 3, 3, 1, 2, 1, 2,
        5, 5, 3, 

...

        2, 4, 2, 3, 3, 3, 2, 3], device='cuda:0') 

current correct is: 5305
current total is: 15000
model prediction:  tensor([1, 3, 3, 2, 2, 3, 2, 3, 3, 2, 4, 2, 3, 2, 2, 2, 3, 2, 3, 3, 4, 5, 2, 1,
        2, 1, 2, 2, 2, 2, 2, 3, 1, 3, 2, 2, 2, 2, 3, 1, 3, 3, 1, 2, 1, 2, 1, 2,
        3, 2, 1, 2, 2, 2, 3, 2, 1, 2, 3, 3, 3, 2, 2, 2, 3, 2, 2, 3, 2, 2, 1, 1,
        1, 3, 2, 1, 3, 2, 2, 1, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 1, 3, 1, 2, 2, 1,
        1, 2, 2, 2, 1, 2, 3, 3, 2, 1, 1, 2, 3, 4, 1, 1, 3, 1, 2, 1, 2, 2, 3, 4,
        2, 2, 1, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 1, 3, 2, 2, 2, 1, 2,
        2, 1, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 4, 1,
        1, 2, 2, 1, 2, 3, 2, 4, 1, 2, 3, 4, 2, 3, 2, 2, 3, 3, 3, 1, 2, 1, 2, 2,
        1, 2, 2, 3, 2, 2, 2, 3], device='cuda:0')
ground truth: tensor([1, 3, 4, 3, 1, 3, 2, 5, 3, 1, 3, 2, 2, 4, 2, 4, 2, 4, 1, 4, 3, 4, 2, 0,
        2, 2, 2, 2, 3, 2, 4, 3, 5, 3, 1, 2, 3, 1, 1, 3, 4, 3, 2, 2, 2, 1, 2, 1,
        3, 2, 4,

        3, 2, 3, 3, 5, 1, 3, 1], device='cuda:0') 

current correct is: 5709
current total is: 16200
model prediction:  tensor([2, 2, 5, 2, 3, 2, 2, 2, 2, 2, 2, 3, 1, 1, 3, 4, 2, 2, 3, 2, 3, 2, 3, 3,
        2, 3, 3, 1, 3, 2, 3, 1, 3, 1, 2, 1, 2, 2, 2, 2, 1, 2, 3, 2, 2, 5, 2, 3,
        3, 2, 3, 2, 3, 3, 3, 2, 2, 2, 3, 2, 2, 2, 1, 2, 2, 1, 2, 3, 4, 3, 3, 2,
        2, 2, 2, 1, 3, 2, 1, 3, 5, 1, 2, 2, 3, 1, 1, 1, 2, 2, 2, 1, 2, 3, 3, 3,
        2, 2, 2, 2, 2, 3, 2, 2, 1, 2, 3, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 3, 2,
        2, 2, 1, 2, 2, 2, 2, 4, 2, 3, 2, 2, 2, 1, 3, 3, 3, 1, 2, 2, 1, 2, 1, 2,
        3, 3, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 1, 3, 1, 3, 2, 1, 3, 2, 2, 2, 2, 2,
        2, 2, 2, 1, 2, 2, 3, 1, 2, 1, 2, 2, 1, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 4, 3, 2], device='cuda:0')
ground truth: tensor([2, 2, 4, 3, 2, 5, 0, 3, 3, 1, 4, 4, 1, 1, 4, 3, 2, 3, 5, 2, 4, 1, 4, 3,
        0, 1, 4, 3, 2, 5, 3, 1, 2, 4, 5, 0, 2, 1, 5, 1, 1, 2, 4, 1, 1, 3, 4, 2,
        0, 3, 2,

        3, 1, 2, 2, 4, 2, 1, 4], device='cuda:0') 

current correct is: 6111
current total is: 17400
model prediction:  tensor([2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 4, 3, 3, 1, 3, 2,
        2, 3, 2, 2, 2, 1, 2, 2, 1, 1, 1, 3, 2, 1, 2, 2, 2, 3, 1, 2, 3, 1, 3, 3,
        2, 4, 2, 2, 2, 3, 3, 1, 2, 3, 1, 1, 2, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 2, 2, 3, 2, 3, 2, 2, 1, 2, 2, 1, 3, 1, 2, 2, 2, 3, 3, 2,
        1, 2, 3, 2, 3, 1, 1, 1, 4, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2,
        2, 2, 3, 3, 2, 3, 2, 1, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 1, 1, 1, 2, 2, 2,
        1, 2, 1, 3, 3, 2, 2, 2, 3, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 3, 2, 2, 2, 5,
        1, 2, 2, 2, 1, 2, 3, 2, 2, 3, 2, 4, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2,
        3, 2, 2, 3, 2, 2, 3, 3], device='cuda:0')
ground truth: tensor([5, 1, 2, 2, 1, 3, 5, 4, 4, 3, 0, 2, 2, 0, 4, 3, 1, 2, 4, 2, 1, 0, 2, 1,
        2, 2, 3, 0, 4, 0, 2, 5, 1, 4, 1, 4, 1, 3, 4, 1, 3, 3, 2, 3, 3, 1, 1, 2,
        3, 4, 3,

        2, 4, 2, 5, 1, 3, 3, 3], device='cuda:0') 

current correct is: 6534
current total is: 18600
model prediction:  tensor([2, 3, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 3, 1, 2, 1, 1, 2, 3, 2, 1, 3, 3, 4,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 2, 2, 2, 2, 1, 2, 3, 3, 3, 1, 3, 3, 1, 4, 3,
        3, 2, 2, 4, 3, 3, 2, 3, 2, 3, 3, 2, 2, 3, 1, 3, 2, 2, 2, 2, 2, 2, 3, 2,
        3, 2, 2, 3, 1, 2, 2, 2, 1, 1, 2, 2, 3, 2, 2, 1, 3, 1, 1, 2, 2, 2, 3, 2,
        1, 3, 2, 1, 1, 2, 2, 3, 3, 2, 2, 2, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 1, 3,
        4, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 1, 3, 4, 2, 2, 3, 1, 2, 2, 2, 2, 2, 2,
        2, 1, 1, 1, 2, 1, 3, 2, 2, 2, 2, 3, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 3, 2,
        3, 3, 2, 1, 2, 2, 2, 2, 1, 3, 1, 2, 1, 3, 4, 1, 2, 2, 2, 2, 1, 2, 2, 2,
        4, 2, 2, 2, 3, 3, 2, 3], device='cuda:0')
ground truth: tensor([2, 1, 2, 1, 5, 4, 1, 2, 1, 1, 1, 1, 2, 1, 3, 2, 1, 3, 3, 2, 2, 1, 2, 3,
        3, 4, 1, 4, 2, 4, 1, 4, 3, 3, 2, 2, 3, 1, 1, 2, 3, 4, 1, 2, 3, 2, 3, 3,
        1, 2, 1,

        1, 5, 3, 3, 4, 3, 3, 2], device='cuda:0') 

current correct is: 6931
current total is: 19800
model prediction:  tensor([4, 2, 2, 2, 2, 1, 2, 2, 3, 2, 2, 2, 3, 2, 3, 3, 3, 2, 3, 3, 1, 2, 2, 2,
        2, 3, 1, 1, 2, 2, 2, 3, 3, 2, 1, 2, 1, 3, 3, 2, 2, 5, 2, 2, 2, 3, 3, 5,
        2, 2, 3, 1, 3, 3, 3, 3, 2, 3, 4, 1, 2, 1, 1, 2, 3, 2, 1, 3, 2, 2, 3, 2,
        1, 3, 3, 2, 2, 3, 2, 3, 3, 2, 1, 2, 2, 1, 2, 2, 3, 2, 2, 1, 2, 2, 3, 2,
        2, 3, 2, 2, 1, 3, 3, 2, 2, 3, 2, 1, 2, 1, 3, 1, 3, 3, 3, 3, 3, 3, 2, 2,
        1, 2, 2, 2, 1, 2, 3, 3, 2, 2, 2, 5, 2, 1, 2, 1, 2, 2, 2, 2, 3, 1, 2, 3,
        3, 2, 2, 1, 2, 1, 1, 2, 3, 3, 2, 2, 3, 1, 2, 3, 1, 2, 2, 1, 3, 3, 1, 2,
        1, 2, 2, 2, 2, 2, 2, 3, 2, 1, 1, 2, 2, 3, 2, 1, 3, 2, 3, 2, 1, 2, 1, 4,
        2, 3, 2, 2, 2, 2, 3, 1], device='cuda:0')
ground truth: tensor([3, 2, 3, 2, 1, 1, 5, 2, 4, 3, 2, 4, 4, 3, 3, 3, 3, 5, 2, 2, 2, 4, 2, 0,
        2, 2, 3, 1, 1, 1, 2, 3, 4, 3, 1, 1, 1, 3, 4, 3, 2, 5, 1, 4, 4, 4, 1, 2,
        2, 2, 5,

In [60]:
for parameters in classifier_net.parameters():
    print(parameters)

Parameter containing:
tensor([[-0.3574, -0.6545, -0.6100,  ..., -0.4404, -0.5575, -0.5853],
        [-0.5219, -0.3637, -0.6106,  ..., -0.6108, -0.6863, -0.4711],
        [-0.6223, -0.5310, -0.4871,  ..., -0.5467, -0.4555, -0.4192],
        ...,
        [-0.8006, -0.6071, -0.5944,  ..., -0.6193, -0.5181, -0.4576],
        [-0.5060, -0.7041, -0.7436,  ..., -0.4418, -0.6517, -0.6329],
        [-0.4919, -0.6297, -0.6702,  ..., -0.5448, -0.5800, -0.6554]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-0.5513, -0.5248, -0.7560, -0.7356, -0.7909, -0.6806, -0.8225, -0.5897,
        -0.7311, -0.5106, -0.6309, -0.5120, -0.8615, -0.7562, -0.7832, -0.5334,
        -0.6108, -0.5723, -0.5278, -0.7827, -0.7936, -0.7378, -0.7447, -0.5563,
        -0.7947, -0.7492, -0.5666, -0.4840, -0.5301, -0.7228, -0.5533, -0.4620,
        -0.7195, -0.6769, -0.6868, -0.5166, -0.7448, -0.5902, -0.5947, -0.7817,
        -0.5918, -0.4671, -0.5231, -0.7449, -0.7588, -0.6344, -0.6841, -0.6849

In [37]:
torch.save(classifier_net, './singlelayerclassifier_params.pt')